#### 1.子公司数据归属地转换

In [1]:
import pandas as pd
import numpy as np

In [2]:
company = pd.read_excel('../../file/subject/chapter3/company_all.xlsx')
len(company)

57432

#### 1.1抽出一部分作为样本进行处理

In [9]:
company.iloc[78:81,:][['维度','经度','地址']]

,维度,经度,地址
78,NaN,NaN,湖南省澧县王家厂镇建设街居委会渔场直属村008号
79,31.317361,121.610042,中国（上海）自由贸易试验区新灵路118号1202A室
80,23.370128,103.395561,库尔勒市人民西路瑞丰商场


#### 1.2填充没有经纬度的行

In [10]:
import requests
import json
import re

def company_lat_lng(x):
    if x['维度'] == np.nan:
        address = x['地址']
        url = 'http://api.map.baidu.com/geocoding/v3/?address=' + address + '&output=json&ak=VYt6ipe4cG9t1CpURQr58ulXep2pcpY6&callback=showLocation'
        data = requests.get(url).text
        js = re.match('.*?\((.*?)\).*?',data).group(1)
        js = json.loads(js)['result']['location']
        return js

In [47]:
company1 = company.iloc[78:81,:][['维度','经度','地址']]

In [14]:
company1.apply(company_lat_lng,axis=1)

78    None
79    None
80    None
dtype: object

In [29]:
#全都没有获取数据，排查发现表格中的Nan并非numpy中的nan
company1['维度'][78] == np.nan

False

In [37]:
#将空值全部替换为0
company2 = company1.fillna(0).copy()

In [39]:
company2['维度'][78]  == 0

True

In [45]:
def company_lat_lng(x):
    if x['维度'] == 0:
        address = x['地址']
        url = 'http://api.map.baidu.com/geocoding/v3/?address=' + address + '&output=json&ak=VYt6ipe4cG9t1CpURQr58ulXep2pcpY6&callback=showLocation'
        data = requests.get(url).text
        js = re.match('.*?\((.*?)\).*?',data).group(1)
        js = json.loads(js)['result']['location']
        return js

In [46]:
#成功判断并获取数据
company2.apply(company_lat_lng,axis=1)

78    {'lng': 111.53989881872957, 'lat': 29.78702709...
79                                                 None
80                                                 None
dtype: object

#### 1.3优化代码

In [48]:
import requests
import json
import re

def company_lat_lng(x):
    try:
        if x['维度'] == 0:
            address = x['地址']
            url = 'http://api.map.baidu.com/geocoding/v3/?address=' + address + '&output=json&ak=VYt6ipe4cG9t1CpURQr58ulXep2pcpY6&callback=showLocation'
            data = requests.get(url).text
            js = re.match('.*?\((.*?)\).*?',data).group(1)
            js = json.loads(js)['result']['location']
            return js
        else:
            return(x[0])
    except:
        print(x['地址'])

In [50]:
company2.apply(company_lat_lng,axis=1)

78    {'lng': 111.53989881872957, 'lat': 29.78702709...
79                                              31.3174
80                                              23.3701
dtype: object

In [51]:
company.fillna(0,inplace=True)

In [53]:
company['lat_lng'] = company[['维度','地址']].apply(company_lat_lng,axis=1)

西昌路
汉沽区营城西街
广中西路775号
江油市太白东路江电路
炉头村
皂市镇
长虹北路
护林路
泗洲镇
武进路456号
0
东汉阳路296号
河南北路369号4－24室
保定路427号
保定路417号
望江西路1号
河北省容城县锦泰花园B区商务7#-05
月湖堤街379号
玉泉南路17号
黄浦大街88号
四平路428号
城东南路13号
四川北路2261号8楼B座
珞璜镇
龙舟坪镇
万荣路949号
泽当镇三湘大道6号
太仓市大连西路66号
巴城镇城北西路2666号2幢
羊市塔
鄂前旗上海庙镇
浙江省宁波市大榭开发区南岗商住4#楼201室
南伞镇
中城镇
环城东路
城中心镇
清波路10号
电厂经济开发区
寺坪镇
建设路
镇平路
汪集镇
古田四路21号
古田四路23号
双桥街8号
清河路
中山北路2550号
太仓市娄东街道洛阳东路81号
黑龙江省佳木斯市向阳区江南社区（江南嘉苑居住组团F楼030#1单元1-3层101室）
天津自贸试验区（东疆保税港区）洛阳道600号海丰物流园3幢2单元-101（天津东疆商务秘书服务有限公司托管第750号）
安庆路219号
北京东路813号
九江路111号
分乡镇
分乡镇
茅坪镇
三斗坪镇
九江路111号
米林县福州东路8号
浙江省温州市苍南县龙港镇中对口村灵宜公路以南、七字山以北（苍南伟明环保能源有限公司办公楼101室）
番禺路667号5F座
贵州六盘水钟山经济开发区红桥西片区纬一路北侧、十二号路东侧、经一路西侧（石桥公寓对面）
解北路37号
鱼洞镇
福州市晋安区象园街道晋连路19号世欧王庄城C-a4地块2#楼45层05办公-9
朝阳北路
洋浦经济开发区吉浦路凯丰城市广场1#综合楼西单元6楼603号房
江城路480号5楼
尤溪镇
曹桥街道
沿山北路
天津自贸试验区（东疆保税港区）洛阳道600号海丰物流园3幢2单元-102（天津东疆商服商务秘书服务有限公司托管第1437号）
清远市清城区人民一路28号清远顺盈时代广场商业中心首层1202、1206、1208、1210号铺，二层2210号铺，三层3210号铺
容城县城关镇南关村南环路50#
大新县桃城镇德天大道470号
河北省廊坊市固安县新兴产业示范区卫星导航产业园40号楼北侧第二层（科技大道北侧，规划三路西侧）
桂林路396号29号楼708室
合肥市高新区红枫路与天智路交口向东200米汽车

In [57]:
company3 = company.iloc[78:81,:][['维度','经度','地址','lat_lng']]
company3

In [72]:
company3.lat_lng = company3.apply(lambda x:company3.lat_lng if x['维度'] == x['lat_lng'] else 0,axis=1)

In [73]:
company3

,维度,经度,地址,lat_lng
78,0.000000,0.000000,湖南省澧县王家厂镇建设街居委会渔场直属村008号,0
79,31.317361,121.610042,中国（上海）自由贸易试验区新灵路118号1202A室,"[0, [0, [0, [0 array(..., dtype=object) array(..."
80,23.370128,103.395561,库尔勒市人民西路瑞丰商场,"[0, [0, [0, [0 array(..., dtype=object) array(..."
